# Issue 132 Notebook Toaletter nära SAT
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb)
* [Issue 132](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132)

Se liknande lösning för Roslagsleden
* nu har vi SAT = wikidata [Q131318799](https://www.wikidata.org/wiki/Q131318799)
* "leden" sitter inte ihop utan varje ö har sitt segment

Jmf dricksvatten [Issue 139](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139) 


Output 
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_28_17_12.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_28_17_12.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_30_04_22.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_30_04_22.html)
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_17_19_29.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_17_19_29.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_16_01.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_16_01.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_23_14.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_23_14.html)




version 0.1
* unary_union --> union_all
* start updating OSM with pictures from https://commons.wikimedia.org/wiki/Category:SAT_Todo
* also count number of seats ie. toilets:number
* Fetch SAT etapper via Wikidata
* Fetch geometries per relation via Overpass (no zip-mismatch)
* Buffer 200 m (Shapely ≥2 with union_all())
* Fetch toilets as nodes/ways/relations via nwr[...] out center;
* Count toilets via toilets:number (fallback male/female/unisex → else 1)
* Join to nearest etapp, build summary (CSV)
* Build a Folium map with etapper, 200 m buffer, and toilet markers
* Export toilets within 200 m as GeoJSON + CSV and save the map HTML

In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-08-19 23:14:41


### New code 

In [2]:
# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import os, re, requests, html
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, MultiLineString, Point, mapping
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup
from datetime import datetime

# =========================
# 1) Hämta SAT-etapper via Wikidata
# =========================
print("🔍 Hämtar SAT-etapper från Wikidata...")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
  OPTIONAL { ?item wdt:P706 ?island. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append({
        "id": r["osmid"]["value"],
        "label": r.get("itemLabel", {}).get("value", ""),
        "island": r.get("islandLabel", {}).get("value", "")
    })
osm_ids = [e['id'] for e in etapper]
print(f"✅ Hittade {len(osm_ids)} etapper med OSM-relationer")

# =========================
# 2) Hämta geometrier per relation via Overpass
# =========================
print("📡 Hämtar geometrier från Overpass (per relation)...")
overpass_url = "http://overpass-api.de/api/interpreter"
geom_per_rel = {}
all_lines = []

for rel_id in osm_ids:
    q = f"""
    [out:json][timeout:60];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    r = requests.post(overpass_url, data={"data": q})
    if r.status_code != 200:
        print(f"⚠️ Fel för relation {rel_id}: {r.text[:200]}...")
        continue
    rel_geoms = []
    for el in r.json().get("elements", []):
        if el.get("type") == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            if len(coords) >= 2:
                line = LineString(coords)
                rel_geoms.append(line)
                all_lines.append(line)
    if rel_geoms:
        geom_per_rel[rel_id] = rel_geoms

if not all_lines:
    raise ValueError("Inga geometrier hittades från OSM-relationer kopplade via Wikidata.")

# Bygg MultiLineString/LineString per etapp
meta_rows, geom_rows = [], []
for meta in etapper:
    geoms = geom_per_rel.get(meta["id"])
    if not geoms:
        print(f"⚠️ Saknar geometri för {meta['label']} (rel {meta['id']}) – hoppar över.")
        continue
    meta_rows.append(meta)
    geom_rows.append(MultiLineString(geoms) if len(geoms) > 1 else geoms[0])

gdf_trail = gpd.GeoDataFrame(geometry=all_lines, crs="EPSG:4326")
meta_gdf = gpd.GeoDataFrame(meta_rows, geometry=geom_rows, crs="EPSG:4326")
print(f"🧭 Etapper med geometri: {len(meta_gdf)}")

# =========================
# 3) Buffert 200 m + ring 200–400 m
# =========================
print("🧮 Skapar 200 m-buffert och 200–400 m-ring...")
trail_utm = gdf_trail.to_crs(3006)
buffer_utm_200 = trail_utm.buffer(200)
buffer_utm_400 = trail_utm.buffer(400)

# Unionera respektive buffert till en geometri
buffer_utm_200_u = buffer_utm_200.unary_union
buffer_utm_400_u = buffer_utm_400.unary_union

# Ring = 400m minus 200m
ring_utm = buffer_utm_400_u.difference(buffer_utm_200_u)

# Tillbaka till WGS84
buffer_union = gpd.GeoSeries([buffer_utm_200_u], crs=3006).to_crs(4326).iloc[0]
ring_union = gpd.GeoSeries([ring_utm], crs=3006).to_crs(4326).iloc[0]

# =========================
# 4) Hämta toaletter (nwr + out center) i bbox
# =========================
def _parse_int(v):
    if v is None:
        return None
    m = re.search(r"\d+", str(v))
    return int(m.group()) if m else None

def toilets_count_from_tags(tags: dict) -> int:
    n = _parse_int(tags.get("toilets:number"))
    if n is not None:
        return n
    parts = [
        _parse_int(tags.get("male:toilets")),
        _parse_int(tags.get("female:toilets")),
        _parse_int(tags.get("unisex:toilets")),
    ]
    parts = [p for p in parts if p is not None]
    if parts:
        return sum(parts)
    return 1

bbox = gdf_trail.total_bounds  # [minx, miny, maxx, maxy]
q_toilets = f"""
[out:json][timeout:60];
nwr["amenity"="toilets"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out center;
"""
print("🚽 Hämtar toaletter (nwr) från Overpass...")
r = requests.post(overpass_url, data={"data": q_toilets})
elements = r.json().get("elements", [])
toilets = []
for el in elements:
    tags = el.get("tags", {})
    typ = el.get("type")
    if typ == "node":
        lon, lat = el["lon"], el["lat"]
    else:
        center = el.get("center")
        if not center:
            continue
        lon, lat = center["lon"], center["lat"]
    toilets.append({
        "geometry": Point(lon, lat),
        "tags": tags,
        "id": el["id"],
        "osm_type": typ,
        "toilets_num": toilets_count_from_tags(tags),
    })

gdf_toilets = gpd.GeoDataFrame(toilets, crs="EPSG:4326")
print(f"✅ Hittade {len(gdf_toilets)} toalett-objekt inom bbox")

# =========================
# 5) Dela upp: inom ≤200 m och i ringen 200–400 m
# =========================
in_0_200 = gdf_toilets[gdf_toilets.geometry.covered_by(buffer_union)]
in_200_400 = gdf_toilets[gdf_toilets.geometry.covered_by(ring_union)]
print(f"✅ {len(in_0_200)} toalett-objekt inom/vid 200 m")
print(f"✅ {len(in_200_400)} toalett-objekt inom 200–400 m")

# =========================
# 6) Närmaste etapp per kategori
# =========================
meta_utm = meta_gdf.to_crs(3006)
toilets_utm_0_200 = in_0_200.to_crs(3006)
toilets_utm_200_400 = in_200_400.to_crs(3006)

joined_0_200 = gpd.sjoin_nearest(
    toilets_utm_0_200,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

joined_200_400 = gpd.sjoin_nearest(
    toilets_utm_200_400,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

# =========================
# 7) Summary för ≤200 m
# =========================
summary = (
    joined_0_200.assign(toilets_num=joined_0_200["toilets_num"].fillna(1))
    .groupby(["label", "island"], as_index=False)
    .agg(
        sites=("geometry", "count"),
        toilets_total=("toilets_num", "sum"),
        avg_distance_m=("distance_m", "mean"),
    )
    .assign(avg_toilets_per_site=lambda df: df["toilets_total"] / df["sites"])
    .sort_values(["toilets_total", "sites"], ascending=[False, False])
)
print("📊 Summary (≤200 m):")
print(summary.head(1000))

# =========================
# 8) Spara filer + grundkarta
# =========================
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
os.makedirs("../kartor", exist_ok=True)

summary_csv = f"../kartor/sat_toaletter_summary_{timestamp}.csv"
summary.to_csv(summary_csv, index=False)

toilets_geojson_0_200 = f"../kartor/sat_toaletter_inrange_0_200_{timestamp}.geojson"
toilets_csv_0_200 = f"../kartor/sat_toaletter_inrange_0_200_{timestamp}.csv"
in_0_200[["id", "osm_type", "toilets_num", "tags", "geometry"]].to_file(toilets_geojson_0_200, driver="GeoJSON")
in_0_200.drop(columns="geometry").to_csv(toilets_csv_0_200, index=False)

toilets_geojson_200_400 = f"../kartor/sat_toaletter_inrange_200_400_{timestamp}.geojson"
toilets_csv_200_400 = f"../kartor/sat_toaletter_inrange_200_400_{timestamp}.csv"
in_200_400[["id", "osm_type", "toilets_num", "tags", "geometry"]].to_file(toilets_geojson_200_400, driver="GeoJSON")
in_200_400.drop(columns="geometry").to_csv(toilets_csv_200_400, index=False)

# Bygg karta
center = gdf_trail.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=9, control_scale=True)

# Etapper
colors = [
    "blue","green","purple","orange","darkred","cadetblue","lightgray","darkblue",
    "darkgreen","pink","lightblue","lightgreen","gray","black","beige","lightred"
]
for i, row in meta_gdf.reset_index(drop=True).iterrows():
    color = colors[i % len(colors)]
    popup = f"<b>{html.escape(row['label'])}</b><br>Ö: {html.escape(row['island'])}"
    folium.GeoJson(
        data=mapping(row.geometry),
        name=row["label"],
        style_function=lambda x, c=color: {"color": c, "weight": 3}
    ).add_child(folium.Popup(popup, max_width=350)).add_to(m)

# Buffert-lager (200 m och 200–400 m)
folium.GeoJson(
    data=mapping(buffer_union),
    name="200 m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

folium.GeoJson(
    data=mapping(ring_union),
    name="Ring 200–400 m",
    style_function=lambda x: {'fillColor': '#ffa500', 'color': '#ffa500', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

# =========================
# 9) Commons helpers (FIX: support Category and avoid double-encoding)
# =========================
def _quote_commons(title: str) -> str:
    # keep existing % escapes to avoid double-encoding already-encoded titles
    return requests.utils.quote(title, safe=':/%')

def commons_title_to_page(title: str) -> str:
    """
    Returnera korrekt Commons-sida (File:/Image:/Category:/etc).
    - Om 'Category:' -> /wiki/Category:...
    - Om File:/Image: -> /wiki/File:...
    - Om ingen prefix -> anta File:
    """
    if not title:
        return None
    t = title.strip()
    low = t.lower()
    if low.startswith(("category:", "file:", "image:")):
        return f"https://commons.wikimedia.org/wiki/{_quote_commons(t)}"
    # default: tolkas som fil
    return f"https://commons.wikimedia.org/wiki/{_quote_commons('File:' + t)}"

def commons_title_to_filepath(title: str, thumb_width=400) -> str | None:
    """
    Thumbnail-url via Special:FilePath — GÄLLER BARA filer.
    Om titeln är en Category, returnera None (ingen direkt bild).
    """
    if not title:
        return None
    t = title.strip()
    low = t.lower()
    if low.startswith("category:"):
        return None
    if not low.startswith(("file:", "image:")):
        t = "File:" + t
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{_quote_commons(t)}?width={thumb_width}"

# Wikidata P18 → Commons thumbnail (oförändrat i sak)
_wikidata_image_cache = {}
def wikidata_p18_thumb(qid: str, thumb_width=400) -> str | None:
    if not qid:
        return None
    qid = qid.strip()
    if qid in _wikidata_image_cache:
        return _wikidata_image_cache[qid]
    try:
        url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
        data = requests.get(url, timeout=15).json()
        ent = data.get("entities", {}).get(qid, {})
        p18 = ent.get("claims", {}).get("P18", [])
        if not p18:
            _wikidata_image_cache[qid] = None
            return None
        filename = p18[0]["mainsnak"]["datavalue"]["value"]
        img_url = commons_title_to_filepath(filename, thumb_width=thumb_width)
        _wikidata_image_cache[qid] = img_url
        return img_url
    except Exception:
        _wikidata_image_cache[qid] = None
        return None

def best_image_url_from_tags(tags: dict) -> str | None:
    """
    Prioritet: image=* URL → image=* (File/Image title) → wikimedia_commons=* (om fil) → wikidata P18
    OBS: Om wikimedia_commons börjar med Category: returnera ingen bild (endast länk i popup).
    """
    if not tags:
        return None
    # 1) image=*
    if "image" in tags and str(tags["image"]).strip():
        first = str(tags["image"]).split(";")[0].strip()
        if first.lower().startswith(("http://", "https://")):
            return first
        # tolka som commons-titel
        return commons_title_to_filepath(first)
    # 2) wikimedia_commons=*
    if "wikimedia_commons" in tags and str(tags["wikimedia_commons"]).strip():
        title = str(tags["wikimedia_commons"]).strip()
        thumb = commons_title_to_filepath(title)  # None if Category
        if thumb:
            return thumb
    # 3) wikidata → P18
    if "wikidata" in tags and str(tags["wikidata"]).strip():
        return wikidata_p18_thumb(tags["wikidata"])
    return None

# =========================
# 10) Taggar & QA
# =========================
BASIC_TAGS = [
    "amenity", "access", "opening_hours", "fee", "wheelchair",
    "toilets:number", "unisex", "drinking_water", "changing_table",
    "toilet:paper", "indoor", "operator", "website", "source"
]

def missing_tags(tags: dict) -> list:
    missing = []
    tags = tags or {}
    for k in BASIC_TAGS:
        if k == "amenity":
            if tags.get("amenity") != "toilets":
                missing.append("amenity=toilets")
            continue
        v = tags.get(k)
        if v is None or str(v).strip() == "":
            missing.append(k)
    if "unisex" in missing and (("male" in tags or "male:toilets" in tags) and ("female" in tags or "female:toilets" in tags)):
        try:
            missing.remove("unisex")
        except ValueError:
            pass
    return missing

def build_basic_table(joined_df):
    tag_cols = [
        "access","opening_hours","fee","wheelchair","toilets:number","unisex",
        "drinking_water","changing_table","toilet:paper","indoor","operator","website","source",
        "wikidata","wikimedia_commons","image"
    ]
    rows = []
    for _, r in joined_df.to_crs(4326).iterrows():
        tags = r.get("tags", {}) or {}
        commons_title = str(tags.get("wikimedia_commons") or "").strip() or None
        commons_page_url = commons_title_to_page(commons_title) if commons_title else None

        img_url = best_image_url_from_tags(tags)
        miss = missing_tags(tags)
        row = {
            "id": r["id"],
            "osm_type": r["osm_type"],
            "lat": r.geometry.y,
            "lon": r.geometry.x,
            "label": r.get("label", ""),
            "island": r.get("island", ""),
            "distance_m": round(float(r.get("distance_m", 0)), 1),
            "toilets_num": int(r.get("toilets_num", 1)),
            "image_url": img_url,
            "missing_tags": ", ".join(miss),
            "wikimedia_commons_title": commons_title,
            "wikimedia_commons_url": commons_page_url
        }
        for k in tag_cols:
            row[k] = (tags.get(k) if isinstance(tags, dict) else None)
        rows.append(row)
    return pd.DataFrame(rows)

# =========================
# 11) Bygg tabeller och lager för ≤200 m och 200–400 m
# =========================
basic_df_0_200 = build_basic_table(joined_0_200)
basic_df_200_400 = build_basic_table(joined_200_400)

basic_csv_0_200 = f"../kartor/sat_toaletter_basic_tags_0_200_{timestamp}.csv"
basic_df_0_200.to_csv(basic_csv_0_200, index=False)
basic_csv_200_400 = f"../kartor/sat_toaletter_basic_tags_200_400_{timestamp}.csv"
basic_df_200_400.to_csv(basic_csv_200_400, index=False)

print("🧾 Basic CSV (≤200 m):", basic_csv_0_200)
print("🧾 Basic CSV (200–400 m):", basic_csv_200_400)

def add_markers(df, feature_group, marker_color):
    for _, r in df.iterrows():
        osm_url = f"https://www.openstreetmap.org/{'node' if r['osm_type']=='node' else 'way' if r['osm_type']=='way' else 'relation'}/{r['id']}"
        img_html = f'<div style="margin:6px 0"><img src="{html.escape(str(r["image_url"]))}" referrerpolicy="no-referrer" style="max-width:280px; height:auto; display:block;"/></div>' if r.get("image_url") else ""
        miss_html = f"<div style='color:#b00; margin-top:4px'><b>Saknade taggar:</b> {html.escape(str(r['missing_tags']))}</div>" if r.get("missing_tags") else ""
        commons_html = ""
        if r.get("wikimedia_commons_url"):
            title = r.get("wikimedia_commons_title") or "Wikimedia Commons"
            commons_html = f'<div style="margin-top:4px">📷 <a href="{html.escape(str(r["wikimedia_commons_url"]))}" target="_blank">{html.escape(str(title))}</a></div>'
        kv = []
        for k in ["opening_hours","fee","wheelchair","access","toilets:number","unisex","drinking_water","changing_table"]:
            v = r.get(k)
            if pd.notna(v) and str(v).strip() != "":
                kv.append(f"{k}={html.escape(str(v))}")
        kv_html = ("<div>" + "<br>".join(kv) + "</div>") if kv else ""
        popup_html = f"""
        <div style="font-size:13px; line-height:1.4">
          <b><a href="{osm_url}" target="_blank">OSM ({r['osm_type']}) {int(r['id'])}</a></b><br>
          Etapp: <b>{html.escape(str(r.get('label') or ''))}</b> (Ö: {html.escape(str(r.get('island') or ''))})<br>
          Avstånd: ~{r['distance_m']} m<br>
          Antal toaletter: <b>{int(r['toilets_num'])}</b>
          {img_html}
          {kv_html}
          {commons_html}
          {miss_html}
        </div>
        """
        Marker(
            location=[r["lat"], r["lon"]],
            popup=Popup(popup_html, max_width=320),
            icon=Icon(color=marker_color, icon="info-sign")
        ).add_to(feature_group)

# ≤200 m lager
toilets_pic_fg_0_200 = FeatureGroup(name="Toaletter ≤200 m (bild, Commons, saknade taggar)")
add_markers(basic_df_0_200, toilets_pic_fg_0_200, marker_color="darkblue")
toilets_pic_fg_0_200.add_to(m)

# 200–400 m lager
toilets_pic_fg_200_400 = FeatureGroup(name="Toaletter 200–400 m (bild, Commons, saknade taggar)")
add_markers(basic_df_200_400, toilets_pic_fg_200_400, marker_color="orange")
toilets_pic_fg_200_400.add_to(m)

LayerControl(collapsed=False).add_to(m)

# =========================
# 12) Spara karta + output
# =========================
map_html = f"../kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_{timestamp}.html"
m.save(map_html)

print("✅ Klart!")
print(f"• Summary CSV (≤200 m): {summary_csv}")
print(f"• Toilets GeoJSON ≤200 m: {toilets_geojson_0_200}")
print(f"• Toilets CSV ≤200 m: {toilets_csv_0_200}")
print(f"• Toilets GeoJSON 200–400 m: {toilets_geojson_200_400}")
print(f"• Toilets CSV 200–400 m: {toilets_csv_200_400}")
print(f"• Basic CSV ≤200 m: {basic_csv_0_200}")
print(f"• Basic CSV 200–400 m: {basic_csv_200_400}")
print(f"• Karta: {map_html}")


🔍 Hämtar SAT-etapper från Wikidata...
✅ Hittade 20 etapper med OSM-relationer
📡 Hämtar geometrier från Overpass (per relation)...
🧭 Etapper med geometri: 20
🧮 Skapar 200 m-buffert och 200–400 m-ring...


/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_22615/3790676905.py:95: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  buffer_utm_200_u = buffer_utm_200.unary_union
/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_22615/3790676905.py:96: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  buffer_utm_400_u = buffer_utm_400.unary_union


🚽 Hämtar toaletter (nwr) från Overpass...
✅ Hittade 721 toalett-objekt inom bbox
✅ 96 toalett-objekt inom/vid 200 m
✅ 9 toalett-objekt inom 200–400 m
📊 Summary (≤200 m):
            label     island  sites  toilets_total  avg_distance_m  \
10    SAT Nåttarö    Nåttarö     11             25       72.232629   
1    SAT Finnhamn   Finnhamn     16             17       29.151231   
2   SAT Fjärdlång  Fjärdlång     10             14       50.783603   
4      SAT Grinda     Grinda     12             12       31.417178   
0     SAT Arholma    Arholma      9             11       39.197231   
12       SAT Rånö       Rånö      6              8       32.477367   
14        SAT Utö        Utö      7              7       17.529076   
5    SAT Ingmarsö   Ingmarsö      5              5       24.822614   
13   SAT Sandhamn     Sandön      4              4       16.953942   
7        SAT Lidö       Lidö      3              3       47.205404   
8        SAT Möja       Möja      3              3       17.

In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-08-19 23:14:57
Total time elapsed: 15.51 seconds
